In [4]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-secret-manager "pinecone-client[grpc]"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.23.0 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.5 which is incompatible.
google-cloud-datastore 1.15.5 requires protobuf<4.0.0dev, but you have protobuf 4.25.5 which is incompatible.
google-cloud-language 1.3.2 requires protobuf<4.0.0dev, but you have protobuf 4.25.5 which is incompatible.
google-cloud-pubsub 2.23.1 requires grpcio<2.0d

In [23]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


In [2]:
import os

import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
from pinecone import PodSpec, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Set API Key
PINECONE_API_KEY = ""  # @param {type:"string"}

pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
# Index Configs
INDEX_NAME = "movie-time-hk"  # @param {type:"string"}

# Choose a distance metric
DISTANCE_METRIC = (
    "cosine"  # @param ["cosine", "euclidean", "dotproduct"] {allow-input: true}
)

EMBEDDING_DIMENSION_SIZE = 768  # @param {"type":"number","placeholder":"768"}

CLOUD_PROVIDER = "aws"  # @param ["gcp", "aws", "azure"] {allow-input: true}

CLOUD_REGION = "us-east-1"  # @param {type:"string"}


# Create the index
pc.create_index(
    name=INDEX_NAME,
    dimension=EMBEDDING_DIMENSION_SIZE,
    metric=DISTANCE_METRIC,
    spec=ServerlessSpec(cloud=CLOUD_PROVIDER, region=CLOUD_REGION),
    deletion_protection="disabled",
)

In [5]:
SECRET_ID = "movie-time"  # @param {type:"string"}

In [7]:
from google.cloud import secretmanager

client = secretmanager.SecretManagerServiceClient()

# Create the secret.
secret = client.create_secret(
    parent=client.common_project_path(PROJECT_ID),
    secret_id=SECRET_ID,
    secret=secretmanager.Secret(
        replication=secretmanager.Replication(
            automatic=secretmanager.Replication.Automatic()
        )
    ),
)

# Add API key to the secret payload.
secret_version = client.add_secret_version(
    parent=secret.name,
    payload=secretmanager.SecretPayload(data=PINECONE_API_KEY.encode("UTF-8")),
)

print(f"Created secret and added first version: {secret_version.name}")

Created secret and added first version: projects/240822342204/secrets/movie-time-hk/versions/1


In [8]:
project_numbers = !gcloud projects list --filter="PROJECT_ID={PROJECT_ID}" --format="value(PROJECT_NUMBER)"
PROJECT_NUMBER = project_numbers[0]

SERVICE_ACCOUNT = f"service-{PROJECT_NUMBER}@gcp-sa-vertex-rag.iam.gserviceaccount.com"

## Create a RAG corpus

In [5]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

In [11]:
vector_db = rag.Pinecone()

DISPLAY_NAME = "movie-time-hk"  # @param  {type:"string"}

rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

Created RAG Corpus resource: projects/240822342204/locations/us-central1/ragCorpora/4611686018427387904


In [15]:
PINECONE_INDEX_NAME = "movie-time-hk"  # @param  {type:"string"}

vector_db = rag.Pinecone(index_name=PINECONE_INDEX_NAME, api_key=secret_version.name)

updated_rag_corpora = rag.update_corpus(
    corpus_name=rag_corpus.name, vector_db=vector_db
)
print(f"Updated RAG Corpus: {rag_corpus.name}")

Updated RAG Corpus: projects/240822342204/locations/us-central1/ragCorpora/4611686018427387904


#### Create a RAG Corpus with Pinecone information

In [ ]:
# Name of your created Pinecone Index
PINECONE_INDEX_NAME = ""  # @param  {type:"string"}
# Construct your Pinecone config.
vector_db = rag.Pinecone(index_name=PINECONE_INDEX_NAME, api_key=secret_version.name)

# Name your corpus
DISPLAY_NAME = ""  # @param  {type:"string"}

# Create RAG Corpus
rag_corpus = rag.create_corpus(display_name=DISPLAY_NAME, vector_db=vector_db)
print(f"Created RAG Corpus resource: {rag_corpus.name}")

In [6]:
rag_file = rag.upload_file(
    corpus_name="projects/240822342204/locations/us-central1/ragCorpora/4611686018427387904",
    path="movie_list_for_testing.txt",
    display_name="movie_list_for_testing.txt",
    description="my test",
)
print(f"Uploaded file to resource: {rag_file.name}")

Uploaded file to resource: projects/240822342204/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5307904854105563513


## Add RAG corpus to context 

In [9]:
rag_resource = rag.RagResource(
    rag_corpus="projects/240822342204/locations/us-central1/ragCorpora/4611686018427387904",
)

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
            similarity_top_k=10,
            vector_distance_threshold=0.4,
        ),
    )
)

rag_model = GenerativeModel("gemini-1.5-flash", tools=[rag_retrieval_tool])

In [10]:
GENERATE_CONTENT_PROMPT = "what are the movies now screening"  # @param {type:"string"}

response = rag_model.generate_content(GENERATE_CONTENT_PROMPT)

response.candidates[0].content.parts[0].text

'The movies currently screening are: The Apprentice, Cesium Fallout, Wicked, Anora, The Wild Robot, Ooops2!, The Substance, Love Lies, URANUS2324, Hello,Love,Again, The Monk And The Gun, Four Trails, The Manor, Paris, Texas, Gladiator 2, RED ONE, Possession Street, Wish Comes True, Venom: The Last Dance, Matched, Shutter. \n'

## Cleaning up

Clean up resources created in this notebook.

In [ ]:
delete_rag_corpus = False  # @param {type:"boolean"}

if delete_rag_corpus:
    rag.delete_corpus(name=rag_corpus.name)